Robot Dynamic Model Identification
=======
---

# 1 - Praparing work

## Import libraries

In [1]:
# enable auto-reload modules
%load_ext autoreload
%autoreload 2
#interactive plotting in separate window
%matplotlib qt 
#%matplotlib inline

from sympy import init_printing
init_printing()
    
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]

import numpy as np
import sympy
from robot_def import RobotDef
from kinematics import Geometry
from dynamics import Dynamics
from trajectory_optimization import TrajOptimizer
from trajectory_optimization import TrajPlotter
from utils import new_sym
from utils import save_data, load_data
import time

## Folder dir for saving and loading files

In [2]:
# model_name = 'test_psm_long2'
#model_name = 'psm_no_off_minert'
model_name = 'psm_complete'

# model_name ='test'
# model_name = 'mtm_3links'
# model_name = 'mtm'
#model_name = 'mtm_spring'
# model_name = 'mtm_3links_parallel'
# model_name = 'mtm_4links_parallel'

model_folder = 'data/' + model_name + '/model/'

---
# 2 - Robot modelling

## Create joint variables and define their relations

## Robot geometry definition in following order

| Joint number | prev link | succ links | $a$ | $\alpha$ | $d$ | $\theta$ | link inertia | motor inertia | friction |
|--------------|-----------|------------|-----|----------|-----|----------|--------------|---------------|---------|

In [26]:
q0, q1, q2, q3, q4, q5, q6, q7, q8, q9, q10 = new_sym('q:11')
_pi = sympy.pi

dh = []
springs = []
friction_type = []

if model_name =='test':
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, False),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, False)]
    q = q1
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    springs = [tau_s]
    friction_type = ['Coulomb', 'viscous', 'offset']
elif model_name == 'mtm':
    # define paralelogram coordinate relation
    q3 = -q2 + q8
    q9 = -q8 + q2
    q10 = q8 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [5],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   4,  [6],    0,     _pi/2,  0,        q5,       True,  False, True),
          (6,   5,  [7],    0,     -_pi/2, 0,        q6+_pi/2, True,  False, True),
          (7,   6,  [],     0,     -_pi/2, 0,        q7+_pi,   True,  False, True),
          (8,   1,  [9],    0,     -_pi/2, 0,        q8+_pi,   True,  False, True),
          (9,   8,  [10],   0.1,   0,      0,        q9-_pi/2, True,  False, True),
          (10,  9,  [],     0.279, 0,      0,        q10,      False, False, True)]

    q = q5
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180.0*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    #springs = [tau_s]
    
    friction_type = ['Coulomb', 'viscous', 'offset']
    print("MTM")
elif model_name == 'mtm_spring':
    # define paralelogram coordinate relation
    q3 = -q2 + q8
    q9 = -q8 + q2
    q10 = q8 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [5],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   4,  [6],    0,     _pi/2,  0,        q5,       True,  False, True),
          (6,   5,  [7],    0,     -_pi/2, 0,        q6+_pi/2, True,  False, True),
          (7,   6,  [],     0,     -_pi/2, 0,        q7+_pi,   True,  False, True),
          (8,   1,  [9],    0,     -_pi/2, 0,        q8+_pi,   True,  False, True),
          (9,   8,  [10],   0.1,   0,      0,        q9-_pi/2, True,  False, True),
          (10,  9,  [],     0.279, 0,      0,        q10,      False, False, True)]

    q = q5
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.0613
    q_o = 23.0/180.0*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    springs = [tau_s]
    
    friction_type = ['Coulomb', 'viscous', 'offset']
    print("MTM")
elif model_name == 'mtm_3links_parallel':
    # define paralelogram coordinate relation
    q3 = -q2 + q4
    q5 = -q4 + q2
    q6 = q4 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,4],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   1,  [5],    0,     -_pi/2, 0,        q4+_pi,   True,  False, True),
          (5,   4,  [6],   0.1,   0,      0,        q5-_pi/2, True,  False, True),
          (6,   5,  [],     0.279, 0,      0,        q6,      False, False, True)]
    friction_type = ['Coulomb', 'viscous']
    #springs = [q1, q2, q3]
elif model_name == 'mtm_4links_parallel':
    # define paralelogram coordinate relation
    q3 = -q2 + q5
    q6 = -q5 + q2
    q7 = q5 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,5],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [0],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   1,  [6],     0,     -_pi/2, 0,        q5+_pi,   True,  False, True),
          (6,   5,  [7],   0.1,   0,      0,        q6-_pi/2, True,  False, True),
          (7,   6,  [],     0.279, 0,      0,        q7,      False, False, True)]
    friction_type = ['Coulomb', 'viscous']
elif model_name == 'mtm_3links':
    dh = [(0,   -1, [1],    0,      0,      0,          0,         False, False, False),
          (1,   0,  [2],    0,      0,      -0.21537,   q1,        True,  False, True),
          (2,   1,  [3],    0,      -_pi/2, 0,          q2+_pi/2,  True,  False, True),
          (3,   2,  [4],    0.279,  0,      0,          q3+_pi/2,  True,  True,  True)]
    # springs = [q1]
    friction_type = ['Coulomb', 'viscous']
elif (model_name == 'test_psm'):
    beta = 0.2908
    beta2 = 0.0767 + beta
    delta = 0.6025
    x = [None]*10
    x[0] = (0,   -1, [1],        0,              0,     0,                       0,      False,  False, False) # Base
    x[1] = (1,   0,  [2],        0,     sympy.pi/2,     0,           q1+sympy.pi/2,       True,  False,  True) # Yaw
    x[2] = (2,   1,  [3, 4],        0, -sympy.pi/2,     0,              sympy.pi/2,       False, False, False) # -- Intermediate
    x[3] = (3,   2,  [5, 6],  -0.0296,           0,     0,    q2-beta - sympy.pi/2,        True,  False, True) # Pitch Back
    x[4] = (4,   2,  [] ,     0.0664 ,           0,     0,    q2-beta - sympy.pi/2,        True,  False, True) # Pitch Front                                    )
    x[5] = (5,   2,  [7],     -0.0296,           0,     0,   q2-beta2 - sympy.pi/2,        False,  False, False)    # Intermediate
    x[6] = (6,   3,  [8],       0.150,           0,     0,    -q2+beta+sympy.pi/2,      True,  False,  True)    # Pitch Bottom
    x[7] = (7,   5,  [],       0.1842,           0,     0,    -q2+beta2+sympy.pi/2,     True,  False,  True)    # Pitch Top
    x[8] = (8,   6,  [9],       0.516,           0,     0,                      q2,     True,  False,  True)    # Pitch End
    x[9] = (9,   8,  [10],      0.043, -sympy.pi/2,-.28809+q3,          sympy.pi/2,     True,  False, True)    # Main Insertion
    dh = x
    friction_type = ['Coulomb', 'viscous', 'offset']     
    
elif (model_name == 'psm_no_off_minert'):
    beta = 0.2908
    beta2 = 0.0767 + beta
    delta = 0.6025
    
    x = [None]*12
    x[0]  = ( 0,  -1,  [1],          0,           0,     0,                       0, False, False, False) # Base
    x[1]  = ( 1,   0,  [2],          0,  sympy.pi/2,     0,           q1+sympy.pi/2,  True, False,  True) # Yaw
    x[2]  = ( 2,   1,  [3],       0, -sympy.pi/2,     0,              sympy.pi/2, False, False, False) # -- Intermediate
    x[3]  = ( 3,   2,  [4, 7], -0.0296,       0,     0,    q2-beta - sympy.pi/2,  True, False,  True) # Pitch Back                                  )
    x[4]  = ( 4,   3,  [5],      0.150,           0,     0,     -q2+beta+sympy.pi/2,  True, False,  True)    # Pitch Bottom
    x[5]  = ( 5,   4,  [6],      0.516,           0,     0,                      q2,  True, False,  True)    # Pitch End
    x[6]  = ( 6,   5, [9],      0.043, -sympy.pi/2,-.28809+q3,          sympy.pi/2,  True, False,  True)    # Main Insertion
    x[7]  = ( 7,   3, [8],          0,           0,     0,         beta+sympy.pi/2, False, False, False) # Intermediate Counterweight
    x[8]  = ( 8,  7,   [],       -0.1, +sympy.pi/2, delta*q3,                    0,  True, False,  True) # Counterweight
    x[9] = (9,   6, [10],          0,           0,0.4162,          -sympy.pi/2+q4, False,  True,  True) # Tool Roll 
    x[10] = (10,  9, [11],          0, -sympy.pi/2,     0,          -sympy.pi/2+q5, False,  True,  True) # Tool Pitch
    x[11] = (11,  10, [12,13],  0.0091, -sympy.pi/2,     0,          -sympy.pi/2+q6, False,  True,  True) # Tool Yaw
    #x[12] = (12,  11, [],            0,           0,     0,                     -q7, False, False,  True) # Gripper1
    #x[13] = (13,  11, [],            0,           0,     0,                      q7, False,  True,  True) # Gripper2
    dh = x
    friction_type = ['Coulomb', 'viscous']
    
elif (model_name == 'psm_complete'):
    beta = 0.2908
    beta2 = 0.0767 + beta
    delta = 0.6025
    
    x = [None]*15
    x[0]  = ( 0,  -1,  [1],          0,           0,     0,                       0, False, False, False) # Base
    x[1]  = ( 1,   0,  [2],          0,  sympy.pi/2,     0,           q1+sympy.pi/2,  True, False,  True) # Yaw
    x[2]  = ( 2,   1,  [3, 4, 5],       0, -sympy.pi/2,     0,              sympy.pi/2, False, False, False) # -- Intermediate
    x[3]  = ( 3,   2,  [6, 10], -0.0296,       0,     0,    q2-beta - sympy.pi/2,  True, False,  True) # Pitch Back
    x[4]  = ( 4,   2,  [] ,     0.0664,           0,     0,    q2-beta - sympy.pi/2,  True, False,  True) # Pitch Front                                    )
    x[5]  = ( 5,   2,  [7],    -0.0296,           0,     0,   q2-beta2 - sympy.pi/2, False, False, False)    # Intermediate
    x[6]  = ( 6,   3,  [8],      0.150,           0,     0,     -q2+beta+sympy.pi/2,  True, False,  True)    # Pitch Bottom
    x[7]  = ( 7,   5,   [],     0.1842,           0,     0,    -q2+beta2+sympy.pi/2,  True, False,  True)    # Pitch Top
    x[8]  = ( 8,   6,  [9],      0.516,           0,     0,                      q2,  True, False,  True)    # Pitch End
    x[9]  = ( 9,   8, [12],      0.043, -sympy.pi/2,-.28809+q3,          sympy.pi/2,  True, False,  True)    # Main Insertion
    x[10] = (10,   3, [11],          0,           0,     0,         beta+sympy.pi/2, False, False, False) # Intermediate Counterweight
    x[11] = (11,  10,   [],       -0.1, +sympy.pi/2, delta*q3,                    0,  True, False,  True) # Counterweight
    
    x[12] = (12,   9, [13],          0,           0,0.4162,          -sympy.pi/2+q4, False,  True,  True) # Tool Roll 
    x[13] = (13,  12, [14],          0, -sympy.pi/2,     0,          -sympy.pi/2+q5, False,  True,  True) # Tool Pitch
    x[14] = (14,  13, [15,16],  0.0091, -sympy.pi/2,     0,          -sympy.pi/2+q6, False,  True,  True) # Tool Yaw
    #x[15] = (15,  14, [],            0,           0,     0,                     -q7, False, False,  True) # Gripper1
    #x[16] = (16,  15, [],            0,           0,     0,                      q7, False,  True,  True) # Gripper2
    dh = x
    friction_type = ['Coulomb', 'viscous', 'offset']
else:
    raise Exception("No robot name matches " + name)
print(friction_type)
robot_def = RobotDef(dh,
                     springs=springs,
                     dh_convention='mdh',
                     friction_type=friction_type)

['Coulomb', 'viscous', 'offset']


In [27]:
robot_def.coordinates_joint_type

['R', 'R', 'P', 'R', 'R', 'R']

In [23]:
robot_def.bary_params

## Create kinematics chain

In [5]:
geom = Geometry(robot_def)

In [42]:
angle = [0, 0, 0, 0, 0, 0, 0]
geom.draw_geom(angle)

## Create dynamics

In [7]:
%time dyn = Dynamics(robot_def, geom)

Calculating Lagrangian...
Calculating the link kinetic energy of 1/14
Calculating the link kinetic energy of 3/14
Calculating the link kinetic energy of 4/14
Calculating the link kinetic energy of 6/14
Calculating the link kinetic energy of 7/14
Calculating the link kinetic energy of 8/14
Calculating the link kinetic energy of 9/14
Calculating the link kinetic energy of 11/14
Calculating joint torques...
tau of q1
tau of q2
tau of q3
tau of q4
tau of q5
tau of q6
Adding frictions, motor rotor inertia and springs...
Calculating gregressor...
('input_vars', (q1, q2, q3, q4, q5, q6, dq1, dq2, dq3, dq4, dq5, dq6, ddq1, ddq2, ddq3, ddq4, ddq5, ddq6))
Calculating M, C and G...
Calculating base parameter...
[ 87  86  85 100  84  97  99  98  59  82  67  69  68  89  24 105 111 107
 113 115 109  31  11  39   6 110 114  38 106  12  90 108 112  10 104  36
  88  46  72  71  27  47   7  19  33  60  15  66  29  30  65  26  63  79
  83  61 101 103  48  70  80 102  77  52  23  32  34  73  28  14  16  7

In [8]:
robot_def.bary_params

In [9]:
sympy.Matrix(dyn.base_param)

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                   

In [11]:
from data import RobotModel

# Data to be saved
robot_model = RobotModel(dyn)

# Save
save_data(model_folder, model_name, robot_model)
print('Saved {} parameters'.format(len(robot_model.base_param)))

Saved 38 parameters


---
# 3 - Excitation trajectory optimization

In [4]:
# model_name = 'test_psm_long2'
model_name = 'psm_no_off_minert'
# model_name ='test'
# model_name = 'mtm_3links'
# model_name = 'mtm'
#model_name = 'mtm_spring'
#model_name = 'mtm_3links_parallel'
# model_name = 'mtm_4links_parallel'

Load Data

In [12]:
from utils import load_data
model_folder = 'data/' + model_name + '/model/'
robot_model = load_data(model_folder, model_name)

## Create trajectory optimizer object, with H matrix, foourier order, base frequency, joint constraints and cartisian constraints as input

In [14]:
trajectory_name = 'one'
# trajectory_name = 'two_normalized'

In [15]:
from numpy import deg2rad

optimal_trajectory_folder = 'data/optimal_trajectory/'
trajectory_folder = 'data/' + model_name +'/optimal_trajectory/'

base_freq = 0.1
fourier_order = 6

joint_constraints = []
cartesian_constraints = []

if model_name == 'mtm' or model_name == 'mtm_spring':
#     joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q4, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q5, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q6, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q7, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q8, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]
#     joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q4, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q5, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q8, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]

    joint_constraints = [(q1, deg2rad(-57), deg2rad(29), deg2rad(-160), deg2rad(160)),
                         (q2, deg2rad(-10), deg2rad(60), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-30), deg2rad(30), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-90), deg2rad(195), -6*np.pi, 6*np.pi),
                         (q5, deg2rad(-87), deg2rad(180), -6*np.pi, 6*np.pi),
                         (q6, deg2rad(-40), deg2rad(38), -6*np.pi, 6*np.pi),
                         (q7, deg2rad(-460), deg2rad(450), -10*np.pi, 10*np.pi),
                         (q8, deg2rad(-9), deg2rad(39), deg2rad(-150), deg2rad(150))]

#     cartesian_constraints = [(3,'max',1,0.4,1),
#                              (3,'min',-1,-1,-1)]
elif model_name == 'mtm_3links':
    joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
                         (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
                         (q3, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]
    
    cartesian_constraints = [(1,'max',1,0.4,1),
                             (1,'min',-1,-1,-1),
                             (2,'max',1,0.4,1),
                             (2,'min',-1,-1,-1)]
    
    # MTM STUFF
#jnt_cnst = [(q1, -0.698132, 1.13446, -2, 2),(q2, -0.261799, 0.872665, -2, 2), (q3,-0.172665, 0.610865, -2, 2)]
#jnt_cnst = [(q1, -1.5, 1.5, -2, 2),(q2, -1.5, 1.5, -2, 2), (q3,-1.5, 1.5, -2, 2)]    

#crt_cnst = [(4, 'max', -0.283, 0.279, -0.25137), (4, 'min', -0.5, -0.321, -0.39137)]
#crt_cnst = [(3, 'max', -0.300, 0.3, -0.26137), (3, 'min', -0.5, -0.45, -0.53137)]
elif model_name == 'mtm_3links_parallel' or model_name == 'mtm_3links_parallel_offset':
    joint_constraints = [(q1, deg2rad(-60), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q2, deg2rad(-12), deg2rad(62), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-32), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-9), deg2rad(39), deg2rad(-360), deg2rad(360))]
elif model_name == 'mtm_4links_parallel':
    joint_constraints = [(q1, deg2rad(-60), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q2, deg2rad(-12), deg2rad(62), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-32), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-90), deg2rad(195), -2*np.pi, 2*np.pi),
                         (q5, deg2rad(-9), deg2rad(39), deg2rad(-360), deg2rad(360))]
elif model_name == 'psm':
    pass
elif model_name == 'test_psm_long2':
    joint_constraints = [(q1, -1.4, 1.4, -2, 2),
                         (q2, -0.5, 0.8, -2, 2),
                         (q3,0.05,0.23,-0.4,0.4),
                         (q4,-1.5,1.5,-0.4,0.4),
                         (q5,-1.5,1.5,-0.4,0.4), 
                         (q6,-1.5,1.5,-0.4,0.4),
                         (q7,0.05, 0.75,-0.4,0.4)]
    
elif model_name == 'psm_complete':
    x = 2
    joint_constraints = [(q1, -1.4, 1.4, -2, 2),
                         (q2, -0.7, 0.7, -2, 2),
                         (q3,0.065,0.23,-0.4,0.4),
                         (q4,-1.5,1.5,-x,x),
                         (q5,-1.5,1.5,-x,x), 
                         (q6,-1.5,1.5,-x,x)]
    
else:
    raise Exception("No robot name matches " + model_name)
    
traj_optimizer = TrajOptimizer(robot_model, fourier_order, base_freq,
                               joint_constraints=joint_constraints,
                               cartesian_constraints = cartesian_constraints)
                                                                                                 
                                                                                                                     
traj_optimizer.optimize()


joint constraint number: 6
cartesian constraint number: 0
constraint number: 24
frames_constrained: []
cond: 512.52078306
cond: 512.520844001
cond: 512.520725379
cond: 512.520751893
cond: 512.520781026
cond: 512.520775195
cond: 512.520765493
cond: 512.520785784
cond: 512.520662598
cond: 512.52071993
cond: 512.520762259
cond: 512.520778621
cond: 512.520799199
cond: 512.520803849
cond: 512.520844162
cond: 512.520863116
cond: 512.520846731
cond: 512.520845768
cond: 512.520829077
cond: 512.520839388
cond: 512.520841436
cond: 512.520683924
cond: 512.520742352
cond: 512.520767187
cond: 512.520794566
cond: 512.520792488
cond: 512.520786902
cond: 512.520775697
cond: 512.520779804
cond: 512.520786342
cond: 512.520781766
cond: 512.52078167
cond: 512.520781598
cond: 512.520782768
cond: 512.520805033
cond: 512.520795304
cond: 512.520795711
cond: 512.520795698
cond: 512.520788522
cond: 512.52078742
cond: 512.52078306
cond: 512.52078306
cond: 512.52078306
cond: 512.52078306
cond: 512.52078306
cond: 

cond: 182.38905937
cond: 182.389114681
cond: 182.389143524
cond: 182.389166161
cond: 182.389196889
cond: 182.389193926
cond: 182.389062358
cond: 182.389150606
cond: 182.389182521
cond: 182.389189529
cond: 182.389184641
cond: 182.38919062
cond: 182.389189557
cond: 182.389376323
cond: 182.389282987
cond: 182.389236214
cond: 182.389204682
cond: 182.389190343
cond: 182.389189449
cond: 182.389183089
cond: 182.389188618
cond: 182.389182667
cond: 182.389186119
cond: 182.389186743
cond: 182.389186358
cond: 182.389186142
cond: 182.3891815
cond: 182.38918153
cond: 182.38918625
cond: 182.389186281
cond: 182.389184366
cond: 182.389185482
cond: 182.389183768
cond: 182.389183768
cond: 182.389183768
cond: 182.389183768
cond: 182.389183768
cond: 182.389183768
cond: 182.389183768
cond: 182.389183768
cond: 182.389183768
cond: 182.389183768
cond: 182.389183768
cond: 182.389183768
cond: 182.389183768
cond: 182.389183768
cond: 182.389183768
cond: 182.389183768
cond: 182.389183768
cond: 182.389183768
cond: 

cond: 446.822209282
cond: 446.822214016
cond: 446.822187564
cond: 446.822184294
cond: 446.822197731
cond: 446.822218162
cond: 446.822243569
cond: 446.822236769
cond: 446.822225994
cond: 446.822220253
cond: 446.822219245
cond: 446.822215411
cond: 446.82220119
cond: 446.822211987
cond: 446.8222196
cond: 446.822228038
cond: 446.822234481
cond: 446.822242023
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
cond: 446.822217216
con

cond: 3541.47062811
cond: 3541.47020554
cond: 3541.47058465
cond: 3541.47056243
cond: 3541.47054511
cond: 3541.47042082
cond: 3541.47038259
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 3541.47063374
cond: 7092.52084097
cond: 7092.52239243
cond: 7092.51539198
cond: 7092.52240971


cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 6338.70612524
cond: 7456.98803668
cond: 7143.89488948
cond: 6313.41335842
cond: 6264.51328537
cond: 6264.51409012
cond: 6264.51571989
cond: 6264.51555853
cond: 6264.51599923
cond: 6264.51146576
cond: 6264.51156953
cond: 6264.5124938
cond: 6264.50948396
cond: 6264.51281959
c

cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 6167.31344079
cond: 7528.99907346
cond: 7831.30613867
cond: 6444.6180357
cond: 6117.60837053
cond: 6117.60986042
cond: 6117.61248161
cond: 6117.61065685
cond: 6117.61123824
cond: 6117.60619222
cond: 6117.60668224
cond: 6117.60761657
cond: 6117.60469712
cond: 6117.60855522
cond: 6117.60873915
cond: 6117.60735946
cond: 6117.60852372
cond: 6117.61086988
c

cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 4356.35764601
cond: 8546.07174575
cond: 5703.07886137
cond: 4573.60727367
cond: 4573.60762053
cond: 4573.61047565
cond: 4573.60560261
cond: 4573.60821204
cond: 4573.60759238
cond: 4573.60597081
cond: 4573.60778331
cond: 4573.60532408
cond: 4573.60816696
cond: 4573.60609613
cond: 4573.60537328
cond: 4573.60828982
cond: 4573.60829996
cond: 4573.59557788
cond: 4573.5963516
cond: 4573.61897497
cond: 4573.60203201
cond: 4573.60548841
cond: 4573.60841448
cond: 4573.60478089
cond: 4573.58997027
cond: 4573.60909717
cond: 4573.61336302
cond: 4573.59939261
cond: 4573.61058848
cond: 4573.60712137
c

cond: 3470.9612487
cond: 3470.9612487
cond: 3470.9612487
cond: 3470.9612487
cond: 3470.9612487
cond: 3470.9612487
cond: 3470.9612487
cond: 3470.9612487
cond: 3470.9612487
cond: 3470.9612487
cond: 3470.9612487
cond: 6442941917.16
cond: 5201028.10871
cond: 10390.4441317
cond: 125.603844738
cond: 410.763232494
cond: 1619.15257454
cond: 2652.16959415
cond: 3136.71396191
cond: 3334.93803536
cond: 3415.20741952
cond: 3447.90982822
cond: 3447.90939094
cond: 3447.90879157
cond: 3447.90909313
cond: 3447.90930878
cond: 3447.90901895
cond: 3447.90945517
cond: 3447.9094835
cond: 3447.90971522
cond: 3447.90980668
cond: 3447.90842235
cond: 3447.90983638
cond: 3447.91024029
cond: 3447.91085602
cond: 3447.90187097
cond: 3447.91389078
cond: 3447.90932858
cond: 3447.90732349
cond: 3447.90538851
cond: 3447.90542065
cond: 3447.90918836
cond: 3447.9239249
cond: 3447.90905777
cond: 3447.90976021
cond: 3447.90711482
cond: 3447.91044241
cond: 3447.90927058
cond: 3447.90991849
cond: 3447.90988529
cond: 3447.90

cond: 6004.41266909
cond: 6004.41266909
cond: 6004.41266909
cond: 6004.41266909
cond: 6004.41266909
cond: 6004.41266909
cond: 6004.41266909
cond: 6640.13150563
cond: 6046.30396145
cond: 6003.75132231
cond: 6003.75072536
cond: 6003.74913073
cond: 6003.751246
cond: 6003.75175763
cond: 6003.75032074
cond: 6003.75103971
cond: 6003.7495789
cond: 6003.75084494
cond: 6003.75079803
cond: 6003.74863035
cond: 6003.75117977
cond: 6003.75078135
cond: 6003.75337972
cond: 6003.73628106
cond: 6003.74892984
cond: 6003.75515931
cond: 6003.74373282
cond: 6003.74242858
cond: 6003.74519551
cond: 6003.74770122
cond: 6003.75567229
cond: 6003.75206338
cond: 6003.75392922
cond: 6003.74652421
cond: 6003.75104
cond: 6003.75106042
cond: 6003.75133093
cond: 6003.75136326
cond: 6003.75124732
cond: 6003.75138203
cond: 6003.75116469
cond: 6003.75132618
cond: 6003.75105739
cond: 6003.75166921
cond: 6003.75129893
cond: 6003.75120298
cond: 6003.7513212
cond: 6003.75133026
cond: 6003.75120917
cond: 6003.75132231
cond: 6

cond: 5918.59093374
cond: 5918.59093374
cond: 5918.59093374
cond: 5918.59093374
cond: 5918.59093374
cond: 5918.59093374
cond: 5918.59093374
cond: 5918.59093374
cond: 3067.41223802
cond: 2115.03681036
cond: 3452.67822063
cond: 3452.67789012
cond: 3452.67795625
cond: 3452.677366
cond: 3452.67866505
cond: 3452.67825297
cond: 3452.67875905
cond: 3452.67856122
cond: 3452.67795176
cond: 3452.67739168
cond: 3452.67719167
cond: 3452.67766388
cond: 3452.67819308
cond: 3452.6789737
cond: 3452.67026171
cond: 3452.68727642
cond: 3452.68304092
cond: 3452.67580288
cond: 3452.67306655
cond: 3452.67389936
cond: 3452.67659554
cond: 3452.68012269
cond: 3452.6757216
cond: 3452.67597233
cond: 3452.67579201
cond: 3452.67861812
cond: 3452.67908109
cond: 3452.67826079
cond: 3452.67821093
cond: 3452.67820961
cond: 3452.6782218
cond: 3452.67820873
cond: 3452.67822444
cond: 3452.67821905
cond: 3452.67820866
cond: 3452.67818
cond: 3452.67821128
cond: 3452.67820542
cond: 3452.67823794
cond: 3452.67822366
cond: 34

In [138]:
reg_norm_mat = traj_optimizer.calc_normalize_mat()

## Plot optimal excitation trajectory

In [139]:
traj_optimizer.calc_frame_traj()

In [140]:
traj_plotter = TrajPlotter(traj_optimizer.fourier_traj,traj_optimizer.frame_traj,
                           traj_optimizer.const_frame_ind, robot_model.coordinates)
traj_plotter.plot_desired_traj(traj_optimizer.x_result)

traj_plotter.plot_frame_traj(True)

## Save trajectory for later use

In [141]:
dof_order_bf_x_norm = (traj_optimizer.fourier_traj.dof, fourier_order,
                       base_freq, traj_optimizer.x_result, reg_norm_mat)
save_data(trajectory_folder, trajectory_name, dof_order_bf_x_norm)

freq = 500
tf = 35 # including 5 seconds' prestable time
traj_optimizer.make_traj_csv(trajectory_folder, trajectory_name, freq, tf)

In [142]:
from IPython.display import HTML, display
import tabulate

table = []
table.append(["joint", 'qo'] +
             ["a"+str(i+1) for i in range(fourier_order)] +
             ["b"+str(i+1) for i in range(fourier_order)])
for i in range(traj_optimizer.fourier_traj.dof):
    line = []
    line.append(robot_model.coordinates[i])
    line += np.round(traj_optimizer.x_result[i*(1 + fourier_order*2): (i+1)*(1 + fourier_order*2)], 4).tolist()
    table.append(line)

display(HTML(tabulate.tabulate(table, tablefmt='html')))

joint,qo,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,b5,b6
q1,-0.2414,0.0067,-0.0019,-0.0071,-0.0007,0.008,0.7491,-0.0006,-0.014,0.0018,-0.0003,-0.0017,2.7823
q2,0.3865,0.0612,-0.0987,0.1611,-0.1103,1.3787,0.2411,-0.026,-0.0576,0.1485,-0.5554,-0.3407,-0.0747
q8,0.2805,0.0946,-0.2565,-0.2301,0.133,0.3449,0.3359,0.0092,-0.2255,0.0786,-0.2744,0.2038,0.0568
q4,0.9192,0.0444,0.7513,1.3374,-0.471,0.1087,-0.6092,0.2326,0.7052,1.8321,-0.339,0.235,-2.7182
q5,1.2311,-0.75,0.8664,-0.1135,1.3938,0.1365,-2.0372,-0.1502,0.2958,-0.1806,0.07,0.8192,-0.2739
q6,-0.0714,0.0277,-0.1577,-0.4567,0.0703,0.1402,-0.2456,0.1268,-0.0566,0.1979,-0.3805,0.8409,0.3349
q7,-2.9053,-0.6684,-0.8079,-3.0967,-3.7475,2.6571,7.4342,-0.8593,3.7604,6.0992,4.2045,-1.9045,-4.7674


---
# Data processing

In [5]:
from identification import load_trajectory_data, diff_and_filt_data, plot_trajectory_data, plot_meas_pred_tau, gen_regressor
from utils import load_data

## Load data

In [6]:
# Names as Arguments

# model_name = 'mtm_3links'
# model_name = 'mtm'
# model_name == 'mtm_3links_parallel'
# model_name == 'mtm_4links_parallel'

# model_name = 'psm_no_off_minert'
model_name = 'psm_complete'

# trajectory_name = 'one'
# trajectory_name = 'two'
trajectory_name = 'one'



In [21]:
# Folders Standard
model_folder = 'data/' + model_name + '/model/'
trajectory_folder = 'data/' + model_name +'/optimal_trajectory/'
results_folder = 'data/' + model_name +'/measured_trajectory/'
results_data_file = results_folder + trajectory_name + '_results.csv'

# Load Results
dof, fourier_order, base_freq, traj_optimizer_result= load_data(trajectory_folder, trajectory_name)


trajectory_sampling_rate = 500
#t, q_raw, tau_raw = load_trajectory_data(results_data_file, trajectory_sampling_rate)
t, q_raw, dq_raw, tau_raw = load_trajectory_data(results_data_file, trajectory_sampling_rate)
dof = q_raw.shape[1]

# Load Robot Model
robot_model = load_data(model_folder,model_name)

(<type 'numpy.ndarray'>, (4989, 18))


## Calculate filter cut-off frequency

In [8]:
fc_mult = 1.3

#radian: fc_mult = 1.3

# this should be loaded from previously saved data
wf = 0.1
order = 6
fc = wf * order * fc_mult
print("cut frequency is {}".format(fc))

cut frequency is 0.78


## Get velocity and acceleration data and filter raw data

In [9]:
t_cut, q_f, dq_f, ddq_f, tau_f, q_raw_cut, tau_raw_cut = diff_and_filt_data(dof, 1.0/trajectory_sampling_rate, t,
                                                                        q_raw, dq_raw, tau_raw, fc, fc, fc, fc)
plot_trajectory_data(t_cut, q_raw_cut, q_f, dq_f, ddq_f, tau_raw_cut, tau_f)

q_raw shape: (4989, 6)
0
1
2
3
4
5
plot shape: 460


---
# Regression

## Ordinary Least Square (OLS)

### Generate regressor matrix for base parameters

In [10]:
base_param_num = robot_model.base_num
H_b_func = robot_model.H_b_func
W_b, tau_s = gen_regressor(base_param_num, H_b_func, q_f, dq_f, ddq_f, tau_f)

In [11]:
print(W_b.shape, tau_s.shape)
print np.linalg.cond(W_b)

((27534, 38), (27534,))
9184.227086771842


In [12]:
xb_ols = np.linalg.lstsq(W_b, tau_s)[0]

/home/radian/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [13]:
#sympy.Matrix.hstack(sympy.Matrix(robot_model.base_param), sympy.Matrix(xb_ols))

from IPython.display import HTML, display
import tabulate

table = [["Base Parameter", "Value"]]

for i in range(robot_model.base_num):
    param_str = str(sympy.Matrix(robot_model.base_param)[i])
    max_disp_len = 50
    line = [param_str if len(param_str) <= max_disp_len else param_str[:max_disp_len] + '...', xb_ols[i]]
    #['%.7s ...'%b if len(str(b)) > 7 else str(b)
    table.append(line)

display(HTML(tabulate.tabulate(table, tablefmt='html')))

Base Parameter,Value
1.0*m9,0.1385591143718002
-0.36018965*l3x - 0.10779909*l3y - 0.36018965*l4x ...,0.04081416638693908
0.10779909*l3x - 0.36018965*l3y + 0.10779909*l4x -...,-0.025336770740975876
1.0*m11,0.48974602620140295
-0.37597504*l1y - 0.37597504*l3z - 0.37597504*l4z ...,0.0214089610483691
1.0*l11x - 0.17891964*l3x + 0.59782515*l3y - 0.178...,0.08200819892704528
1.0*l11z + 0.59782515*l3x + 0.17891964*l3y + 0.597...,-0.08710196332610474
1.0*l11y + 0.62402496*l1y + 0.62402496*l3z + 0.624...,-0.052528539954444516
1.0*l7y,-0.888881863885041
-0.27468079*L3xx + 0.83558474*L3xy + 0.27468079*L3...,0.026686101122252632


## Compare measured torque and predicted torque

### Generate predicted torque

In [14]:
tau_p = np.zeros(tau_f.shape)
tau_ps = W_b.dot(xb_ols)
for i in range(dof):
    tau_p[:, i] = tau_ps[i::dof]
    

### Evaluate regression

In [15]:
var_regression_error_ols = np.linalg.norm(tau_ps - tau_s) / (tau_ps.size - base_param_num)
print("variance of regression error using OLS:")
print(var_regression_error_ols)

std_dev_xb_ols = np.sqrt(np.diag(var_regression_error_ols * np.linalg.inv(W_b.transpose().dot(W_b))))
print("standard deviation of xb using OLS:")
print(std_dev_xb_ols)

pct_std_dev_xb_ols = std_dev_xb_ols / np.abs(xb_ols)
print("percentage of standard deviation of xb using OLS: ")
print(pct_std_dev_xb_ols)

variance of regression error using OLS:
0.0003346149856175889
standard deviation of xb using OLS:
[2.32602834e-04 1.53263907e-04 1.05973785e-04 4.50788462e-04
 1.21695988e-03 1.93026937e-04 1.25090160e-04 2.02177207e-03
 7.63250521e-02 4.94219059e-04 1.95457127e-04 5.25820873e-04
 4.06059320e-03 4.43164433e-03 4.69476629e-04 1.94538206e-03
 5.45390058e-05 1.13470530e-04 8.99603323e-04 5.65294123e-05
 6.30662114e-04 1.05891231e-04 7.38861802e-04 7.12629235e-03
 3.91351491e-02 2.73300169e-04 2.71068247e-04 2.37838541e-03
 3.02470771e-04 3.18484185e-02 2.44755227e-03 3.99324247e-04
 3.83606229e-04 5.01766837e-04 5.21477082e-04 4.93843436e-04
 5.80442942e-04 1.14752798e-01]
percentage of standard deviation of xb using OLS: 
[1.67872633e-03 3.75516445e-03 4.18260820e-03 9.20453537e-04
 5.68434816e-02 2.35375169e-03 1.43613480e-03 3.84890208e-02
 8.58663623e-02 1.85197177e-02 2.05849593e-02 2.42776130e-02
 4.42670924e-02 2.81554258e-03 2.53814771e-03 4.11659874e-01
 3.55917043e-02 4.79215043

### Plot measured torque and predicted torque

In [28]:
#plot_data_num = int(1 / wf * trajectory_sampling_rate)
#plot_meas_pred_tau(t_cut[:plot_data_num], tau_f[:plot_data_num, :], tau_p[:plot_data_num, :])


plot_data_num = int(1 / base_freq * trajectory_sampling_rate)
plot_meas_pred_tau(t_cut[:plot_data_num], tau_f[:plot_data_num, :],
                   tau_p[:plot_data_num, :],
                   robot_def.coordinates_joint_type,
                   robot_model.coordinates)

## Weighted Least Square (WLS)

In [29]:
np.sqrt(np.linalg.norm(tau_f - tau_p, axis=0)/(tau_f.shape[0] - base_param_num))

array([0.0330048 , 0.02794946, 0.0386372 , 0.0030578 , 0.0132746 ,
       0.01322352])

In [30]:
weight = np.max(tau_f, axis=0) - np.min(tau_f, axis=0)
weight
weight[1] /= 1
weight[2] /= 1
weight[3] /= 1
weight

array([2.49872831, 3.02617854, 2.19205591, 0.00661718, 0.1152496 ,
       0.0799544 ])

In [31]:
W_b.shape, tau_s.shape

In [32]:
# repeat the weight to generate a large vecoter for all the data
weights = 1.0/np.tile(weight, W_b.shape[0]/weight.shape[0])

In [33]:
weights.shape

In [34]:
W_b_wls = np.multiply(W_b, np.asmatrix(weights).transpose())
tau_s_wls = np.multiply(tau_s, weights)

In [35]:
xb_wls = np.linalg.lstsq(W_b_wls, tau_s_wls)[0]

/home/radian/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [36]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.base_param), sympy.Matrix(xb_wls))

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                   

In [37]:
tau_p_wls = np.zeros(tau_f.shape)
tau_ps_wls = W_b.dot(xb_wls)
for i in range(dof):
    tau_p_wls[:, i] = tau_ps_wls[i::dof]

In [38]:
plot_meas_pred_tau(t_cut[:plot_data_num], tau_f[:plot_data_num, :], tau_p_wls[:plot_data_num, :])

TypeError: plot_meas_pred_tau() takes exactly 5 arguments (3 given)

In [39]:
np.linalg.norm(tau_f[:plot_data_num, :] - tau_p_wls[:plot_data_num, :], axis=0)\
/ np.linalg.norm(tau_f[:plot_data_num, :], axis=0)

array([0.11256344, 0.06197482, 0.14414638, 0.09384515, 0.42094513,
       0.41675646])

## Convex optimization

### Generate regressor matrix for barycentric parameters

In [35]:
from identification import SDPOpt

bary_param_num = len(robot_model.bary_param)
H_func = robot_model.H_func
W, tau_s = gen_regressor(bary_param_num, H_func, q_f, dq_f, ddq_f, tau_f)

In [36]:
W_cvx = np.multiply(W, np.asmatrix(weights).transpose())
tau_s_cvx = np.multiply(tau_s, weights)

In [37]:
len(robot_model.std_param)

| min mass | max mass | min $r_x$ | max $r_x$ | min $r_y$ | max $r_y$ | min $r_z$ | max $r_z$ | max $F_c$ | max $F_v$ | max $F_o$ |
|----------|----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|--------------|-----------|

In [38]:
sdp_constraints = []
if model_name == 'mtm':
    sdp_constraints = [(0.5, 10, -0.1, 0.1, -0.1, 0.1, -0.1, 0.3, 0.2, 0.2, 0.2),
                      (0.3, 5, -0.1, 0.2, -0.05, 0.05, -0.05, 0.05, 0.2, 0.2, 0.2),
                      (0.1, 3, -0.0, 0.3, -0.02, 0.02, -0.01, 0.01, 0.2, 0.2, 0.2),
                      (0.1, 1, -0.01, 0.01, -0.0, 0.1, -0.1, 0.0, 0.2, 0.2, 0.1),
                      (0.02, 0.3, -0.01, 0.01, -0.05, 0.0, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.2, -0.005, 0.005, -0.00, 0.05, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.01, 0.1, -0.000, 0.000, -0.00, 0.00, -0.00, 0.05, 0.1, 0.1, 0.1),
                      (0.3, 5, -0.10, 0.05, -0.05, 0.05, -0.05, 0.05, 0.1, 0.2, 0.2),
                      (0.05, 0.5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1),
                      (0.0, 0, 0.00, 0.00, -0.00, 0.00, 0.000, 0.00, 0.1, 0.1, 0.0)]
elif model_name == 'mtm_3links':
    sdp_constraints = [(0.5, 10, -0.1, 0.1, -0.1, 0.1, -0.1, 0.3, 0.5, 0.5, 0.5),
                      (0.3, 5, -0.1, 0.2, -0.05, 0.05, -0.05, 0.05, 0.5, 0.5, 0.5),
                      (0.1, 3, -0.0, 0.3, -0.02, 0.02, -0.01, 0.01, 0.1, 0.5, 0.5),
                      (0.1, 1, -0.01, 0.01, -0.0, 0.1, -0.1, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.3, -0.01, 0.01, -0.05, 0.0, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.2, -0.005, 0.005, -0.00, 0.05, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.01, 0.1, -0.000, 0.000, -0.00, 0.00, -0.00, 0.05, 0.1, 0.1, 0.1),
                      (0.3, 5, -0.10, 0.05, -0.05, 0.05, -0.05, 0.05, 0.5, 0.5, 0.5),
                      (0.3, 5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1),
                      (0.3, 5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1)]
    
elif model_name == 'psm_complete':
    sdp_constraints = [(0.5, 1.5, -0.2, -0.01, -0.07, 0.07, -0.15, 0.15, 1, 1, 1), #yaw_link
                      (0.3, 1, -0.1, 0.2, -0.05, 0.05, -0.05, 0.05, 1, 1, 1),     # Intermediate
                      (0.1, 1.5, -0.1, 0.15, -0.02, 0.02, -0.01, 0.01, 1, 1, 1), #Pitch Back
                      (0.1, 0.15, 0.0, 0.15, -0.02, 0.02, -0.01, 0.01, 1, 1, 1), #Pitch Front 
                      (0.02, 1, -0.01, 0.01, -0.05, 0.0, -0.05, 0.0, 0.1, 0.1, 0.1), #Intermediate
                      (0.02, 0.5, 0.0, 0.515, -0.05, 0.02, -0.01, 0.01, 1, 1, 1), #Pitch Bottom
                      (0.01, 0.3, 0.000, 0.515, -0.02, 0.02, -0.01, 0.01, 1, 1, 1), #Pitch Top
                      (0.01, 0.6, -0.2, 0.2, -0.4, 0.01, -0.02, 0.02, 1, 1, 1), #Pitch End
                      (0.01, 0.4, -0.02, 0.02, -0.01, 0.01, -0.01, 0.05, 3, 3, 3), # Main Insertion
                      (0.01, 1, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1), # Intermediate Counterweight
                      (0.0001, 0.4, -0.04, 0.04, -0.01, 0.01, -0.02, 0.03, 1, 1, 1), # Counterweight
                      (0.0001, 1, -1, 1, -1, 1, -1, 1, 1, 1, 1), # Tool Roll
                      (0.0001, 1, -1, 1, -1, 1, -1, 1, 1, 1, 1), # Tool Pitch
                      (0.0001, 1, -1, 1, -1, 1, -1, 1, 1, 1, 1)]# Tool Yaw
else:
    raise Exception("No robot name matches " + model_name)

In [39]:


sdp_opt_std = SDPOpt(W_cvx, tau_s_cvx, robot_model, sdp_constraints)
# sdp_opt_std = SDPOpt(W, tau_s, robot_model, sdp_constraints)
%time sdp_opt_std.solve()

Regressor shape: (27534, 116)
Regressand shape: (27534,)
Creating variables...
Creating optimization objective...
Creating constraints...
Ia12 param108
Ia13 param112
Ia14 param116
Solving problem...
----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 688650, CG tol ~ 1/iter^(2.00)
eps = 1.00e-03, alpha = 1.50, max_iters = 50000, normalize = 1, scale = 1.00
Variables n = 117, constraints m = 27838
Cones:	linear vars: 134
	soc vars: 27536, soc blks: 1
	sd vars: 168, sd blks: 8
Setup time: 3.10e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -na

  9300| 6.28e-01  7.64e-03  5.59e-06  9.99e+03  9.99e+03  1.32e-14  1.73e+02 
  9400| 6.21e-01  6.82e-03  3.64e-06  1.02e+04  1.02e+04  1.35e-14  1.74e+02 
  9500| 6.15e-01  3.66e-03  4.72e-06  1.03e+04  1.03e+04  1.38e-14  1.75e+02 
  9600| 6.08e-01  2.39e-03  4.45e-06  1.05e+04  1.05e+04  1.41e-14  1.77e+02 
  9700| 6.01e-01  4.43e-03  3.80e-06  1.07e+04  1.07e+04  1.44e-14  1.78e+02 
  9800| 5.94e-01  5.77e-03  4.97e-06  1.08e+04  1.08e+04  1.46e-14  1.79e+02 
  9900| 5.87e-01  6.48e-03  3.44e-06  1.10e+04  1.10e+04  1.49e-14  1.80e+02 
 10000| 5.79e-01  4.62e-03  4.70e-06  1.11e+04  1.11e+04  1.52e-14  1.81e+02 
 10100| 5.72e-01  3.14e-03  3.81e-06  1.13e+04  1.13e+04  1.54e-14  1.82e+02 
 10200| 5.64e-01  2.31e-03  3.90e-06  1.14e+04  1.14e+04  1.57e-14  1.83e+02 
 10300| 5.56e-01  3.46e-03  4.30e-06  1.16e+04  1.16e+04  1.59e-14  1.85e+02 
 10400| 5.48e-01  4.88e-03  3.39e-06  1.17e+04  1.17e+04  1.62e-14  1.87e+02 
 10500| 5.40e-01  4.44e-03  4.30e-06  1.18e+04  1.18e+04  1.64e-

 19900| 1.07e-01  3.32e-04  3.44e-07  1.68e+04  1.68e+04  1.31e-14  3.17e+02 
 20000| 1.06e-01  5.34e-04  3.30e-07  1.69e+04  1.69e+04  1.32e-14  3.18e+02 
 20100| 1.05e-01  3.91e-04  3.30e-07  1.69e+04  1.69e+04  1.32e-14  3.19e+02 
 20200| 1.04e-01  3.72e-04  3.10e-07  1.69e+04  1.69e+04  1.32e-14  3.20e+02 
 20300| 1.03e-01  3.60e-04  3.07e-07  1.69e+04  1.69e+04  1.32e-14  3.21e+02 
 20400| 1.02e-01  3.50e-04  2.98e-07  1.70e+04  1.70e+04  1.32e-14  3.22e+02 
 20500| 1.00e-01  6.99e-04  2.83e-07  1.70e+04  1.70e+04  1.33e-14  3.23e+02 
 20600| 9.95e-02  4.73e-04  2.84e-07  1.70e+04  1.70e+04  1.33e-14  3.24e+02 
 20700| 9.85e-02  3.88e-04  2.68e-07  1.70e+04  1.70e+04  1.33e-14  3.25e+02 
 20800| 9.75e-02  5.52e-04  2.63e-07  1.70e+04  1.70e+04  1.33e-14  3.26e+02 
 20900| 9.66e-02  4.03e-04  2.58e-07  1.71e+04  1.71e+04  1.33e-14  3.27e+02 
 21000| 9.57e-02  3.01e-04  2.43e-07  1.71e+04  1.71e+04  1.34e-14  3.28e+02 
 21100| 9.48e-02  4.71e-04  2.44e-07  1.71e+04  1.71e+04  1.34e-

 30500| 5.69e-02  1.45e-03  2.40e-08  1.84e+04  1.84e+04  1.45e-14  4.25e+02 
 30600| 5.67e-02  1.80e-03  1.77e-08  1.84e+04  1.84e+04  1.45e-14  4.26e+02 
 30700| 5.64e-02  1.33e-03  1.72e-08  1.84e+04  1.84e+04  1.46e-14  4.27e+02 
 30800| 5.62e-02  4.48e-04  2.31e-08  1.84e+04  1.84e+04  1.46e-14  4.28e+02 
 30900| 5.60e-02  6.96e-04  1.15e-08  1.85e+04  1.85e+04  1.46e-14  4.29e+02 
 31000| 5.58e-02  9.80e-04  1.33e-08  1.85e+04  1.85e+04  1.46e-14  4.30e+02 
 31100| 5.56e-02  1.16e-03  1.44e-08  1.85e+04  1.85e+04  1.46e-14  4.31e+02 
 31200| 5.54e-02  4.12e-04  1.57e-08  1.85e+04  1.85e+04  1.46e-14  4.34e+02 
 31300| 5.52e-02  6.93e-04  1.89e-08  1.85e+04  1.85e+04  1.46e-14  4.35e+02 
 31400| 5.50e-02  7.67e-04  1.53e-08  1.85e+04  1.85e+04  1.46e-14  4.36e+02 
 31500| 5.47e-02  4.28e-04  1.71e-08  1.85e+04  1.85e+04  1.46e-14  4.37e+02 
 31600| 5.45e-02  3.04e-04  1.60e-08  1.85e+04  1.85e+04  1.46e-14  4.38e+02 
 31700| 5.44e-02  6.68e-04  1.29e-08  1.85e+04  1.85e+04  1.46e-

 41100| 3.84e-02  1.58e-04  3.38e-09  1.92e+04  1.92e+04  1.52e-14  5.37e+02 
 41200| 3.82e-02  1.34e-04  2.84e-09  1.92e+04  1.92e+04  1.52e-14  5.38e+02 
 41300| 3.81e-02  2.51e-04  2.51e-09  1.92e+04  1.92e+04  1.52e-14  5.39e+02 
 41400| 3.80e-02  2.40e-04  3.27e-09  1.92e+04  1.92e+04  1.52e-14  5.39e+02 
 41500| 3.78e-02  1.41e-04  2.18e-09  1.92e+04  1.92e+04  1.52e-14  5.40e+02 
 41600| 3.77e-02  1.39e-04  2.32e-09  1.92e+04  1.92e+04  1.52e-14  5.41e+02 
 41700| 3.75e-02  2.23e-04  1.97e-09  1.92e+04  1.92e+04  1.52e-14  5.42e+02 
 41800| 3.74e-02  1.95e-04  1.57e-09  1.92e+04  1.92e+04  1.52e-14  5.43e+02 
 41900| 3.72e-02  2.34e-04  1.14e-09  1.92e+04  1.92e+04  1.52e-14  5.44e+02 
 42000| 3.71e-02  1.61e-04  1.14e-09  1.92e+04  1.92e+04  1.52e-14  5.45e+02 
 42100| 3.70e-02  1.82e-04  1.11e-09  1.92e+04  1.92e+04  1.52e-14  5.46e+02 
 42200| 3.68e-02  1.95e-04  1.01e-09  1.93e+04  1.93e+04  1.52e-14  5.47e+02 
 42300| 3.67e-02  2.37e-04  5.19e-10  1.93e+04  1.93e+04  1.52e-

## Compare measured torque and predicted torque

### Generate predicted torque

In [40]:
tau_sdp_p = np.zeros(tau_f.shape)
tau_sdp_ps = W.dot(sdp_opt_std.x_result)
for i in range(dof):
    tau_sdp_p[:, i] = tau_sdp_ps[i::dof]

### Plot measured torque and predicted torque

In [41]:
plot_data_num = int(1 / wf * trajectory_sampling_rate)
plot_meas_pred_tau(t_cut[:plot_data_num] - t_cut[0],
                   tau_f[:plot_data_num, :],
                   tau_sdp_p[:plot_data_num, :])
# plot_meas_pred_tau(t_cut, tau_f, tau_sdp_p)

In [42]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.bary_param), sympy.Matrix(sdp_opt_std.x_result))

⎡L1xx     146.492248456821   ⎤
⎢                            ⎥
⎢L1xy     -141.68070080345   ⎥
⎢                            ⎥
⎢L1xz   1.85472162802146e-17 ⎥
⎢                            ⎥
⎢L1yy     248.090250704819   ⎥
⎢                            ⎥
⎢L1yz   -4.62861599181923e-17⎥
⎢                            ⎥
⎢L1zz    0.00387714472533742 ⎥
⎢                            ⎥
⎢ l1x    -0.0218438438064678 ⎥
⎢                            ⎥
⎢ l1y   -0.00786373527225522 ⎥
⎢                            ⎥
⎢ l1z   2.40908501936636e-19 ⎥
⎢                            ⎥
⎢ m₁      1.49999853864434   ⎥
⎢                            ⎥
⎢ Fc₁   0.000980587438144429 ⎥
⎢                            ⎥
⎢ Fv₁    0.0717253759297873  ⎥
⎢                            ⎥
⎢ Fo₁    -0.323602621710236  ⎥
⎢                            ⎥
⎢L3xx    0.00259122993110163 ⎥
⎢                            ⎥
⎢L3xy    -0.002198439766149  ⎥
⎢                            ⎥
⎢L3xz    0.00171473244892783 ⎥
⎢                            ⎥
⎢L3yy   

In [43]:
from identification import barycentric2standard_params

In [44]:
sdp_opt_std.x_result

array([ 1.46492248e+02, -1.41680701e+02,  1.85472163e-17,  2.48090251e+02,
       -4.62861599e-17,  3.87714473e-03, -2.18438438e-02, -7.86373527e-03,
        2.40908502e-19,  1.49999854e+00,  9.80587438e-04,  7.17253759e-02,
       -3.23602622e-01,  2.59122993e-03, -2.19843977e-03,  1.71473245e-03,
        1.89328601e-02,  2.72408181e-03,  1.81274678e-02, -1.51477617e-01,
       -1.43996256e-02, -1.49041824e-02,  1.50007128e+00,  1.04028133e-03,
        3.96173291e-02, -4.88633604e-02,  2.60346305e-03, -7.96475021e-04,
        3.51745984e-04,  3.43128822e-03,  2.75315399e-03,  2.76663078e-03,
       -1.11502661e-04, -2.99067101e-03, -1.37031341e-03,  1.49998053e-01,
        1.04028133e-03,  3.96173291e-02, -4.88633604e-02,  3.52379394e-03,
       -1.22741931e-10, -1.95683809e-10,  1.14412563e+02,  4.94511692e+01,
        2.15138280e+02,  1.94367457e-08, -2.18026188e-04, -1.75575298e-04,
        2.01447270e-02,  1.04028133e-03,  3.96173291e-02,  4.88633604e-02,
        3.52483073e-03, -

In [45]:
x_std = barycentric2standard_params(sdp_opt_std.x_result, robot_model)

In [46]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.std_param), sympy.Matrix(x_std))

⎡I1xx     146.492207231226   ⎤
⎢                            ⎥
⎢I1xy     -141.680586287202  ⎥
⎢                            ⎥
⎢I1xz   1.85437080316714e-17 ⎥
⎢                            ⎥
⎢I1yy     248.089932602168   ⎥
⎢                            ⎥
⎢I1yz   -4.62874228798788e-17⎥
⎢                            ⎥
⎢I1zz    0.00351781647881753 ⎥
⎢                            ⎥
⎢ r1x    -0.0145625767250479 ⎥
⎢                            ⎥
⎢ r1y   -0.00524249528893694 ⎥
⎢                            ⎥
⎢ r1z   1.60605824425911e-19 ⎥
⎢                            ⎥
⎢ m₁      1.49999853864434   ⎥
⎢                            ⎥
⎢ Fc₁   0.000980587438144429 ⎥
⎢                            ⎥
⎢ Fv₁    0.0717253759297873  ⎥
⎢                            ⎥
⎢ Fo₁    -0.323602621710236  ⎥
⎢                            ⎥
⎢I3xx    0.00230492095552235 ⎥
⎢                            ⎥
⎢I3xy   -0.000744361546911101⎥
⎢                            ⎥
⎢I3xz    0.00321976095457358 ⎥
⎢                            ⎥
⎢I3yy   

In [47]:
sdp_constraints

In [48]:
from identification import modify_sdf, add_trans_config
frame = [1]
len(frame)

In [51]:
frame_name = [' ', 'yaw_Link', ' ', 'pitch_back_Link', 'pitch_front_Link', ' ', 'pitch_bottom_Link',
                   'pitch_top_Link', 'pitch_end_Link', 'main_insertion_Link', ' ' , 'counterweight_Link', 'tool_roll_link',
                   'tool_pitch_link', 'tool_yaw_link']

modify_sdf(robot_model, x_std, frame_names = frame_name, filename = 'psm.sdf')
add_trans_config(robot_model, x_std, frame_names = frame_name)

In [89]:
robot_model.frame_num

In [42]:
robot_model.use_inertia

[False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False]

In [43]:
robot_model.use_friction

[False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True]

In [44]:
robot_model.friction_type

['Coulomb', 'viscous', 'offset']

In [ ]:
robot_model.use_inertia